# In this notebook I'll do put all the features together (old and new ones) on the original training set provided by the tutor. <br> For testing, I'll use an uncleaned portion train.csv.

In [10]:
import numpy as np
import pandas as pd
import datetime
import time
from pandas.tseries.holiday import USFederalHolidayCalendar
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

In [3]:
pwd

'C:\\Users\\Petro Ck\\OneDrive\\Task 2\\_petro'

In [9]:
taxi = pd.read_table("C:/Users/Petro Ck/OneDrive/Task 2/data/train.csv", sep = ",")

In [12]:
taxi.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-06-09 21:06:36,2016-06-09 21:13:08,2,0.79,-73.98336,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.0,0.0,0.3,7.3
1,2,2016-06-09 21:06:36,2016-06-09 21:35:11,1,5.22,-73.98172,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.0,0.0,0.3,27.3


In [20]:
x = taxi.iloc[:,0:-1] # independent variables (predictors)
x.head(2) 

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge
0,2,2016-06-09 21:06:36,2016-06-09 21:13:08,2,0.79,-73.98336,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.0,0.0,0.3
1,2,2016-06-09 21:06:36,2016-06-09 21:35:11,1,5.22,-73.98172,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.0,0.0,0.3


In [21]:
y = taxi.iloc[:,-1] # dependent (target) variable
y.head(2)           # array!! Do we want to keep dimensions?

0     7.3
1    27.3
Name: total_amount, dtype: float64

## Create pickup and dropoff

In [29]:
pickup = pd.to_datetime(taxi["tpep_pickup_datetime"])
dropoff = pd.to_datetime(taxi["tpep_dropoff_datetime"])

# New features

### Create Trip duration ###

In [31]:
taxi["trip_duration"] = pickup - dropoff 

In [32]:
taxi["trip_duration"] = taxi["trip_duration"].dt.seconds

### Create average_speed ### 

In [33]:
taxi["average_speed"] = taxi["trip_distance"] / (taxi["trip_duration"] / 3600)

## Create the dummies ##

### Night Trip

This variable is 1 if the ride was between 8pm and 6am.

In [34]:
taxi["night_trip"] = 0

#### Consider pickup
taxi["night_trip"][(pickup.dt.time <= datetime.time(6,0,0)) |
                        (pickup.dt.time >= datetime.time(20,0,0))] = 1

#### Consider also dropoff
taxi["night_trip"][(dropoff.dt.time <= datetime.time(6,0,0)) |
                   (dropoff.dt.time >= datetime.time(20,0,0))] = 1

C:\Users\Petro Ck\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Petro Ck\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### Rush hour

This variables in 1 on weekdays excluding holidays from 4pm to 8pm

In [ ]:
us_holidays = USFederalHolidayCalendar().holidays()

In [ ]:
taxi["rush_hour"] = 1

taxi["rush_hour"][(dropoff.dt.time < datetime.time(16,0,0)) |   
                  (pickup.dt.time > datetime.time(20,0,0)) |    
                  (pickup.dt.dayofweek == 5) |                  
                  (pickup.dt.dayofweek == 6) |
                  (pickup.dt.date.isin(us_holidays.date))] = 0  

## Splitting data

In [27]:
x_train,x_test,y_train,y_test = train_test_split(taxi, y, test_size=0.3)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(7794829, 19) (7794829,)
(3340641, 19) (3340641,)


# Features for pickup and dropoff time #

### Create Trip duration ###

In [ ]:
taxi["trip_duration"] = taxi["tpep_dropoff_datetime"] - taxi["tpep_pickup_datetime"] 

In [6]:
taxi["trip_duration"] = taxi["trip_duration"].dt.seconds